# Tech Meeting 2024 Q1 - Run with(out) `ini` files

The technical meeting that took place on 2024.04.03 related to the Q1 sprints covered the following topics:

1. Running an analysis whilst modifying the `.ini` configuration files.
2. Usage of enumerations within `ra2ce`.
3. Discussion on long term views for `ra2ce` subprojects.
4. Walk-through, on how to add new analyses to the current solution.

This jupyter notebook will covers the first point.

## 1. Running an analysis whilst modifying the `.ini` configuration files

It is entirely possible to generate a ra2ce analysis (or just a network) without having to generate `.ini` files. However, most of the times we will  only want to modify a few properties of a given `network.ini` or `analysis.ini` file.
At the current version `v0.8.1` we can solve both options as follows:

1. Load the `.ini` files and then modify them at our own choice.
2. Fully creating a `Ra2ceHandler` instance based on python objects ( `NetworkConfigData` and `AnalysisConfigData`).

### 1.1. Modify existing configurations from code

Given our most common use case, we want to load a network and analysis and potentially modifying some of its properties. This was already accomplished in the "ra2ce hackathon 2024 Q1".

To achieve it, simply use the regular `Ra2ceHandler` with your desired `ini` files and then access the corresponding `NetworkConfigData` and `AnalysisConfigData` properties. See the example below.

1. Initialize the Ra2ce handler with valid `.ini` files.

In [ ]:
from pathlib import Path

from ra2ce.ra2ce_handler import Ra2ceHandler

# Define the location of our example test data.
_root_dir = Path("data").joinpath("direct_analysis")
assert _root_dir.exists()

_network_file = _root_dir.joinpath("network.ini")
assert _network_file.exists()

_analysis_file = _root_dir.joinpath("analysis.ini")
assert _analysis_file.exists()

# Initialize handler
_handler = Ra2ceHandler(_network_file, _analysis_file)

2. In addition, we can verify if the analysis and network are correctly set.

In [ ]:
from ra2ce.network.network_config_data.network_config_data import NetworkConfigData
from ra2ce.analysis.analysis_config_data.analysis_config_data import AnalysisConfigData

assert isinstance(_handler.input_config.network_config.config_data, NetworkConfigData)
assert isinstance(_handler.input_config.analysis_config.config_data, AnalysisConfigData)

3. Configure the network and analyses

In [ ]:
_handler.configure()

4. Run the analysis.

In [ ]:
_handler.run_analysis()

### 1.2. Create Ra2ce Handler without `ini` files

__DISCLAIMER!__
This functionality will be streamlined as resolution of issue [#389](https://github.com/Deltares/ra2ce/issues/389)

This step, however longer than required, will demonstrate how to emulate the usage of `ini` files. We relate the `.ini` files as:

- `network.ini` to `NetworkConfigData`,
- `analysis.ini` to `AnalysisConfigData`.

Both `ConfigData` items are python objects (`dataclasses`) as such, it is entirely possible to manually manipulate them via code, therefore avoiding the need of definition and usage of their related `ini` files.

1. Initialize the objects representing the `.ini` files.

In [ ]:
from ra2ce.network.network_config_data.network_config_data import NetworkConfigData
from ra2ce.analysis.analysis_config_data.analysis_config_data import AnalysisConfigData

_network_config_data = NetworkConfigData()
_analysis_config_data = AnalysisConfigData()

2. (Optional) Initialize logger.

In [ ]:
from ra2ce.ra2ce_logging import Ra2ceLogger

# Initialize logger.
_output_logger_path = Path("data").joinpath("logging")
if _output_logger_path.exists():
    import shutil
    shutil.rmtree(_output_logger_path)
_output_logger_path.mkdir(parents=True)

Ra2ceLogger(logging_dir=_output_logger_path, logger_name="RA2CE")

3. Put said objects together in the config wrapper.

In [ ]:
# Define network config wrapper
from ra2ce.network.network_config_wrapper import NetworkConfigWrapper

_network_config_wrapper = NetworkConfigWrapper()
_network_config_wrapper.config_data = _network_config_data
_network_config_wrapper.configure()

# Define analysis config wrapper
from ra2ce.analysis.analysis_config_wrapper import AnalysisConfigWrapper

_analysis_config_wrapper = AnalysisConfigWrapper()
_analysis_config_wrapper.config_data = _analysis_config_data
_analysis_config_wrapper.config_data.network = _network_config_wrapper.config_data.network
_analysis_config_wrapper.config_data.origins_destinations = (
    _network_config_wrapper.config_data.origins_destinations
)
_analysis_config_wrapper.graph_files = _network_config_wrapper.graph_files
_analysis_config_wrapper.configure()

# Initialize wrapper
from ra2ce.configuration.config_wrapper import ConfigWrapper

_config_wrapper = ConfigWrapper()
_config_wrapper.analysis_config = _analysis_config_wrapper
_config_wrapper.network_config = _network_config_wrapper

_config_wrapper.configure()

4. Run analysis

In [ ]:
# Run analysis
from ra2ce.runners import AnalysisRunnerFactory

_runner = AnalysisRunnerFactory.get_runner(_config_wrapper)
_runner.run(_config_wrapper.analysis_config)

### 1.3. Create Ra2ce Handler without `ini` files - after #389

In this example the network and analysis configuration are read from file before creation of the `Ra2ceHandler`.
This can be replaced by any other way of creating your configuration, e.g. by creating it from scratch by `NetworkConfigWrapper()` and `AnalysisConfigWrapper()`.
The handler doesn't need file at creation time.
The analysis should yield the same results as `example_direct_damage.ipynb`.
Note that the logfile isn't created.

In [6]:
from pathlib import Path

from ra2ce.analysis.analysis_config_data.analysis_config_data_reader import AnalysisConfigDataReader
from ra2ce.network.network_config_data.network_config_data_reader import NetworkConfigDataReader
from ra2ce.ra2ce_handler import Ra2ceHandler
from ra2ce.runners import AnalysisRunnerFactory

# Initialize configuration (replace this with your own configuration)
_data_dir = Path("../data/direct_analysis")
_network_ini = _data_dir.joinpath("network.ini")
_network = NetworkConfigDataReader().read(_network_ini)
_network.root_path = _data_dir.parent
_network.input_path = _data_dir.joinpath("input")
_network.static_path = _data_dir.joinpath("static")
_network.output_path = _data_dir.joinpath("output")
_analysis_ini = _data_dir.joinpath("analysis.ini")
_analysis = AnalysisConfigDataReader().read(_analysis_ini)

# Initialize handler from configuration
_handler = Ra2ceHandler.from_config(_network, _analysis)

# Configure handler
_handler.configure()

# Run analysis
_runner = AnalysisRunnerFactory.get_runner(_handler.input_config)
_runner.run(_handler.input_config.analysis_config)

c:\Users\klapwij\code\ra2ce\.env\lib\site-packages\osmnx\simplification.py:454: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  merged = utils_graph.graph_to_gdfs(G, edges=False)["geometry"].buffer(tolerance).unary_union
c:\Users\klapwij\code\ra2ce\.env\lib\site-packages\osmnx\simplification.py:501: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids = node_clusters.centroid
100%|██████████| 4092/4092 [00:00<00:00, 272802.43it/s]
C:\Users\klapwij\code\ra2ce\ra2ce\network\networks_utils.py:1859: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

[AnalysisResultWrapper(analysis_result=                                  osmid  oneway   infra_type  reversed  \
 u           v           key                                              
 567924681   567929942   0      44785078    True     tertiary     False   
             1934771035  0     820923407   False     tertiary      True   
 567924688   5639948250  0      44794760   False     tertiary     False   
             5639948249  0      44794760   False     tertiary      True   
 567924689   4724793628  0      44794760   False     tertiary     False   
 ...                                 ...     ...          ...       ...   
 11793125032 11793125033 0    1270107335   False  residential     False   
             11793125031 0    1270107335   False  residential      True   
 11793125033 4095038521  0      44794760   False     tertiary     False   
             5651483641  0      44794760   False     tertiary      True   
             11793125032 0    1270107335   False  residential 